# Predicting CAR Commute Time 

## Import All Packages Used

In [1]:
#Import Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

from pyspark.sql import functions as fn, Row
from pyspark.sql.functions import when, lit
from pyspark import sql
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.ml import feature, regression, evaluation, Pipeline
from pyspark.ml.feature import VectorAssembler,StringIndexer,OneHotEncoder, StandardScaler,QuantileDiscretizer
from pyspark.ml.regression import LinearRegression, RandomForestRegressor, GBTRegressor
from pyspark.ml.tuning import ParamGridBuilder

# Data Processing

In [2]:

#Load data
trip_df = pd.read_csv("/dbfs/FileStore/tables/trippub.csv")
hhpub_df = pd.read_csv("/dbfs/FileStore/tables/hhpub.csv")
per_df = pd.read_csv("/dbfs/FileStore/tables/perpub.csv")
veh_df= pd.read_csv("/dbfs/FileStore/tables/vehpub.csv")

In [3]:
trip_df.loc[trip_df.WTTRDFIN > 0]
hhpub_df.loc[hhpub_df.WTHHFIN > 0]
per_df.loc[per_df.WTPERFIN > 0]
veh_df.loc[veh_df.WTHHFIN >0]

,HOUSEID,VEHID,VEHYEAR,VEHAGE,MAKE,MODEL,FUELTYPE,VEHTYPE,WHOMAIN,OD_READ,HFUEL,VEHOWNED,VEHOWNMO,ANNMILES,HYBRID,PERSONID,TRAVDAY,HOMEOWN,HHSIZE,HHVEHCNT,HHFAMINC,DRVRCNT,HHSTATE,HHSTFIPS,NUMADLT,WRKCOUNT,TDAYDATE,LIF_CYC,MSACAT,MSASIZE,RAIL,URBAN,URBANSIZE,URBRUR,CENSUS_D,CENSUS_R,CDIVMSAR,HH_RACE,HH_HISP,HH_CBSA,SMPLSRCE,WTHHFIN,BESTMILE,BEST_FLG,BEST_EDT,BEST_OUT,HBHUR,HTHTNRNT,HTPPOPDN,HTRESDN,HTEEMPDN,HBHTNRNT,HBPPOPDN,HBRESDN
0,30000007,1,2007,10,49,49032,1,1,3,69000,-1,1,-1,1000,2,3,2,1,3,5,7,3,NC,37,3,1,201608,10,3,1,2,1,1,1,5,3,53,2,2,XXXXX,2,187.314320,14611.926637,1,-1,4,T,50,1500,750,750,20,750,300
1,30000007,2,2004,13,49,49442,1,2,-8,164000,-1,1,-1,2500,2,-8,2,1,3,5,7,3,NC,37,3,1,201608,10,3,1,2,1,1,1,5,3,53,2,2,XXXXX,2,187.314320,4767.090946,3,-1,-1,T,50,1500,750,750,20,750,300
2,30000007,3,1998,19,19,19014,1,1,1,120000,-1,1,-1,900,2,1,2,1,3,5,7,3,NC,37,3,1,201608,10,3,1,2,1,1,1,5,3,53,2,2,XXXXX,2,187.314320,8000.324552,1,-1,-1,T,50,1500,750,750,20,750,300
3,30000007,4,1997,20,19,19021,1,1,2,-88,-1,1,-1,500,2,2,2,1,3,5,7,3,NC,37,3,1,201608,10,3,1,2,1,1,1,5,3,53,2,2,XXXXX,2,187.314320,725.932347,2,-1,-1,T,50,1500,750,750,20,750,300
4,30000007,5,1993,24,20,20481,1,4,2,300000,-1,1,-1,10000,2,2,2,1,3,5,7,3,NC,37,3,1,201608,10,3,1,2,1,1,1,5,3,53,2,2,XXXXX,2,187.314320,12437.658757,1,-1,-1,T,50,1500,750,750,20,750,300
5,30000008,1,2014,3,20,20028,1,1,97,-88,-1,1,-1,8000,2,97,5,1,2,4,8,2,WI,55,2,2,201608,2,2,5,2,4,6,2,3,2,32,1,2,33460,2,69.513032,6099.620939,3,-1,-1,R,5,300,300,150,5,300,300
6,30000008,2,2010,7,20,20024,1,3,2,-88,-1,1,-1,10000,2,2,5,1,2,4,8,2,WI,55,2,2,201608,2,2,5,2,4,6,2,3,2,32,1,2,33460,2,69.513032,12865.611703,2,-1,-1,R,5,300,300,150,5,300,300
7,30000008,3,2008,9,20,20039,1,1,97,-88,-1,1,-1,10000,2,97,5,1,2,4,8,2,WI,55,2,2,201608,2,2,5,2,4,6,2,3,2,32,1,2,33460,2,69.513032,6118.535423,3,-1,-1,R,5,300,300,150,5,300,300
8,30000008,4,2004,13,20,20481,1,4,1,128000,-1,1,-1,10000,2,1,5,1,2,4,8,2,WI,55,2,2,201608,2,2,5,2,4,6,2,3,2,32,1,2,33460,2,69.513032,9777.895446,1,-1,-1,R,5,300,300,150,5,300,300
9,30000012,1,2011,6,58,58037,1,1,1,70115,-1,1,-1,11000,2,1,5,1,1,2,10,1,NY,36,1,1,201607,1,3,3,2,1,3,1,2,1,23,1,2,XXXXX,2,79.419586,4616.917047,1,-1,-1,C,80,17000,17000,5000,60,17000,7000


In [4]:
trip_df1 = trip_df[
    np.abs(trip_df.WTTRDFIN - trip_df.WTTRDFIN.mean()) <= (3*trip_df.WTTRDFIN.std())
]
hhpub_df1 = hhpub_df[
    np.abs(hhpub_df.WTHHFIN - hhpub_df.WTHHFIN.mean()) <= (3*hhpub_df.WTHHFIN.std())
]
per_df1 = per_df[
    np.abs(per_df.WTPERFIN - per_df.WTPERFIN.mean()) <= (3*per_df.WTPERFIN.std())
]
veh_df1 = veh_df[
    np.abs(veh_df.WTHHFIN - veh_df.WTHHFIN.mean()) <= (3*veh_df.WTHHFIN.std())
]

In [5]:
print("hhpub_df before cleaning: ",hhpub_df.shape)
print("hhpub_df1 after cleaning: ",hhpub_df1.shape)
print("per_df before cleaning: ",per_df.shape)
print("per_df1 after cleaning: ",per_df1.shape)
print("trip_df before cleaning: ",trip_df.shape)
print("trip_df1 after cleaning: ",trip_df1.shape)
print("veh_df before cleaning: ",veh_df.shape)
print("veh_df1 after cleaning: ",veh_df1.shape)

hhpub_df before cleaning: (129696, 58)
hhpub_df1 after cleaning: (126638, 58)
per_df before cleaning: (264234, 121)
per_df1 after cleaning: (258022, 121)
trip_df before cleaning: (923572, 105)
trip_df1 after cleaning: (902122, 105)
veh_df before cleaning: (256115, 54)
veh_df1 after cleaning: (250091, 54)

In [6]:
#Way to Import data into spark dataframe on databricks:
#trip_df  = spark.read.format("csv").option("header", "true").load("/FileStore/tables/trippub.csv")
#hhpub_df = spark.read.format("csv").option("header", "true").load("/FileStore/tables/hhpub.csv")
#per_df   = spark.read.format("csv").option("header", "true").load("/FileStore/tables/perpub.csv")
#veh_df   = spark.read.format("csv").option("header", "true").load("/FileStore/tables/vehpub.csv")

In [7]:
#Loading Data into Spark Data Frame
hhpub = spark.createDataFrame(hhpub_df1)
vehpub = spark.createDataFrame(veh_df1)
trippub = spark.createDataFrame(trip_df1)
perpub = spark.createDataFrame(per_df1)

In [8]:
#Function to remove columns containing invalid/NA values:
def rmv_nacols(df):
  dropcol = ['1']
  id = ['HOUSEID', 'VEHID', 'PERSONID']
  #dfcol = redmergeddf.columns
  total = df.count()
  for i in df.columns:
    if (i == id[0]) or (i == id[1]) or (i == id[2]):
      #print(i)
      continue
    else:
      #print(i)
      a = df.where(fn.col(i).isin({-1, -9,-7,-77,-8,-88})).count()
      naval = (a/total)*100
      print('Percentage of NA values is '+ str(naval) + 'in column ' + i)
      if naval > 50.00:
        dropcol.append(i)
  print(dropcol)
  return dropcol




In [9]:
hhnacols = rmv_nacols(hhpub)
print(type(hhnacols))
print(len(hhnacols))
print(hhnacols)

Percentage of NA values is 0.0in column TRAVDAY
Percentage of NA values is 0.0in column SAMPSTRAT
Percentage of NA values is 0.026058529035518568in column HOMEOWN
Percentage of NA values is 0.0in column HHSIZE
Percentage of NA values is 0.0in column HHVEHCNT
Percentage of NA values is 3.3876087746174135in column HHFAMINC
Percentage of NA values is 1.7885626747105925in column PC
Percentage of NA values is 2.8364314028964452in column SPHONE
Percentage of NA values is 4.2633332806898405in column TAB
Percentage of NA values is 8.793569070894993in column WALK
Percentage of NA values is 11.296767163094806in column BIKE
Percentage of NA values is 1.9449138489237037in column CAR
Percentage of NA values is 10.474739019883447in column TAXI
Percentage of NA values is 10.165195281037287in column BUS
Percentage of NA values is 10.428939180972536in column TRAIN
Percentage of NA values is 12.042988676384656in column PARA
Percentage of NA values is 0.7493801228699127in column PRICE
Percentage of NA values is 2.490563653879562in column PLACE
Percentage of NA values is 3.6316113646772687in column WALK2SAVE
Percentage of NA values is 4.526287528230073in column BIKE2SAVE
Percentage of NA values is 4.001958337939639in column PTRANS
Percentage of NA values is 0.0in column HHRELATD
Percentage of NA values is 0.0in column DRVRCNT
Percentage of NA values is 0.0in column CNTTDHH
Percentage of NA values is 0.0in column HHSTATE
Percentage of NA values is 0.0in column HHSTFIPS
Percentage of NA values is 0.0in column NUMADLT
Percentage of NA values is 0.0in column YOUNGCHILD
Percentage of NA values is 0.0in column WRKCOUNT
Percentage of NA values is 0.0in column TDAYDATE
Percentage of NA values is 0.0in column HHRESP
Percentage of NA values is 0.0007896523950157141in column LIF_CYC
Percentage of NA values is 0.0in column MSACAT
Percentage of NA values is 0.0in column MSASIZE
Percentage of NA values is 0.0in column RAIL
Percentage of NA values is 0.0in column URBAN
Percentage of NA values is 0.0in column URBANSIZE
Percentage of NA values is 0.0in column URBRUR
Percentage of NA values is 0.0in column SCRESP
Percentage of NA values is 0.0in column CENSUS_D
Percentage of NA values is 0.0in column CENSUS_R
Percentage of NA values is 0.0in column CDIVMSAR
Percentage of NA values is 0.5266981474754813in column HH_RACE
Percentage of NA values is 0.12397542601746711in column HH_HISP
Percentage of NA values is 0.0in column HH_CBSA
Percentage of NA values is 0.0in column RESP_CNT
Percentage of NA values is 0.5124844043651985in column WEBUSE17
Percentage of NA values is 0.0in column SMPLSRCE
Percentage of NA values is 0.0in column WTHHFIN
Percentage of NA values is 0.08528245866169712in column HBHUR
Percentage of NA values is 0.08528245866169712in column HTHTNRNT
Percentage of NA values is 0.08528245866169712in column HTPPOPDN
Percentage of NA values is 0.08528245866169712in column HTRESDN
Percentage of NA values is 0.08528245866169712in column HTEEMPDN
Percentage of NA values is 0.08528245866169712in column HBHTNRNT
Percentage of NA values is 0.08528245866169712in column HBPPOPDN
Percentage of NA values is 0.08528245866169712in column HBRESDN
['1']
<class 'list'>
1
['1']

In [10]:
#Household DataFrame after removing Columns with invalid/NA values

hhpub1 = reduce(DataFrame.drop, hhnacols, hhpub)

print('Before removing columns with NA values: ', len(hhpub.columns))
print('After removing columns with NA values: ', len(hhpub1.columns))

Before removing columns with NA values: 58
After removing columns with NA values: 58

In [11]:
vehnacols = rmv_nacols(vehpub)
print(type(vehnacols))
print(len(vehnacols))
print(vehnacols)

Percentage of NA values is 0.6685566453810813in column VEHYEAR
Percentage of NA values is 0.6685566453810813in column VEHAGE
Percentage of NA values is 0.695346893730682in column MAKE
Percentage of NA values is 1.760559156467046in column MODEL
Percentage of NA values is 0.12635400714140052in column FUELTYPE
Percentage of NA values is 0.12995269721821257in column VEHTYPE
Percentage of NA values is 0.18153392165251847in column WHOMAIN
Percentage of NA values is 15.829438084537228in column OD_READ
Percentage of NA values is 97.5261004994182in column HFUEL
Percentage of NA values is 0.38066143923611806in column VEHOWNED
Percentage of NA values is 86.38655529387303in column VEHOWNMO
Percentage of NA values is 25.73143375811205in column ANNMILES
Percentage of NA values is 0.0in column HYBRID
Percentage of NA values is 0.0in column TRAVDAY
Percentage of NA values is 0.02359141272576782in column HOMEOWN
Percentage of NA values is 0.0in column HHSIZE
Percentage of NA values is 0.0in column HHVEHCNT
Percentage of NA values is 3.188439408055468in column HHFAMINC
Percentage of NA values is 0.0in column DRVRCNT
Percentage of NA values is 0.0in column HHSTATE
Percentage of NA values is 0.0in column HHSTFIPS
Percentage of NA values is 0.0in column NUMADLT
Percentage of NA values is 0.0in column WRKCOUNT
Percentage of NA values is 0.0in column TDAYDATE
Percentage of NA values is 0.0007997089059582312in column LIF_CYC
Percentage of NA values is 0.0in column MSACAT
Percentage of NA values is 0.0in column MSASIZE
Percentage of NA values is 0.0in column RAIL
Percentage of NA values is 0.0in column URBAN
Percentage of NA values is 0.0in column URBANSIZE
Percentage of NA values is 0.0in column URBRUR
Percentage of NA values is 0.0in column CENSUS_D
Percentage of NA values is 0.0in column CENSUS_R
Percentage of NA values is 0.0in column CDIVMSAR
Percentage of NA values is 0.501417484035811in column HH_RACE
Percentage of NA values is 0.13235182393608727in column HH_HISP
Percentage of NA values is 0.0in column HH_CBSA
Percentage of NA values is 0.0in column SMPLSRCE
Percentage of NA values is 0.0in column WTHHFIN
Percentage of NA values is 1.569828582396008in column BESTMILE
Percentage of NA values is 0.0in column BEST_FLG
Percentage of NA values is 97.85198187859619in column BEST_EDT
Percentage of NA values is 93.09251432478578in column BEST_OUT
Percentage of NA values is 0.08117045395476047in column HBHUR
Percentage of NA values is 0.08117045395476047in column HTHTNRNT
Percentage of NA values is 0.08117045395476047in column HTPPOPDN
Percentage of NA values is 0.08117045395476047in column HTRESDN
Percentage of NA values is 0.08117045395476047in column HTEEMPDN
Percentage of NA values is 0.08117045395476047in column HBHTNRNT
Percentage of NA values is 0.08117045395476047in column HBPPOPDN
Percentage of NA values is 0.08117045395476047in column HBRESDN
['1', 'HFUEL', 'VEHOWNMO', 'BEST_EDT', 'BEST_OUT']
<class 'list'>
5
['1', 'HFUEL', 'VEHOWNMO', 'BEST_EDT', 'BEST_OUT']

In [12]:
#Vehicle DataFrame after removing Columns with invalid/NA values

vehpub1 = reduce(DataFrame.drop, vehnacols, vehpub)

print('Before removing columns with NA values: ', len(vehpub.columns))
print('After removing columns with NA values: ', len(vehpub1.columns))

Before removing columns with NA values: 54
After removing columns with NA values: 50

In [13]:
#This is not requried to run if we are running the next block as results of this block is hardcoded in next block
pernacols = rmv_nacols(perpub)
print(type(pernacols))
print(len(pernacols))
print(pernacols)

Percentage of NA values is 0.18719334010278194in column R_AGE
Percentage of NA values is 8.509351915728116in column EDUC
Percentage of NA values is 0.153862848904357in column R_HISP
Percentage of NA values is 0.031005108091558087in column R_RELAT
Percentage of NA values is 0.09185263272124082in column R_SEX
Percentage of NA values is 0.5472401578160002in column R_RACE
Percentage of NA values is 10.415003371805504in column PRMACT
Percentage of NA values is 52.13702707521064in column PAYPROF
Percentage of NA values is 53.36754230259435in column GT1JBLWK
Percentage of NA values is 53.37335576036152in column WRK_HOME
Percentage of NA values is 53.39544689987675in column WKFTPT
Percentage of NA values is 59.756919952562185in column WRKTRANS
Percentage of NA values is 83.27739495081815in column LSTTRDAY17
Percentage of NA values is 53.390408569811875in column OCCAT
Percentage of NA values is 87.80142778522762in column SCHTYP
Percentage of NA values is 0.30966351706443634in column NWALKTRP
Percentage of NA values is 27.56547891265086in column WALK4EX
Percentage of NA values is 0.08022571718690655in column NBIKETRP
Percentage of NA values is 89.33695576346203in column BIKE4EX
Percentage of NA values is 89.30943873003078in column BIKESHARE
Percentage of NA values is 0.13525978404942213in column PTUSED
Percentage of NA values is 98.10752571486152in column MCUSED
Percentage of NA values is 10.483214609606934in column CARSHARE
Percentage of NA values is 10.478176279542055in column RIDESHARE
Percentage of NA values is 62.970211842401035in column CARRODE
Percentage of NA values is 59.846447202176556in column TIMETOWK
Percentage of NA values is 63.00935579136663in column NOCONG
Percentage of NA values is 98.58151630481122in column PUBTIME
Percentage of NA values is 54.19189061397866in column WRKTIME
Percentage of NA values is 59.80226492314608in column WKRMHM
Percentage of NA values is 53.45551929680414in column FLEXTIME
Percentage of NA values is 93.0680329584299in column WKFMHMXX
Percentage of NA values is 88.48082721628388in column SCHTRN1
Percentage of NA values is 88.48043965243274in column SCHTRN2
Percentage of NA values is 10.587469285564795in column DELIVER
Percentage of NA values is 0.051933556053359795in column MEDCOND
Percentage of NA values is 90.45352721860927in column MEDCOND6
Percentage of NA values is 0.09766609048840796in column HEALTH
Percentage of NA values is 0.1844803931447706in column PHYACT
Percentage of NA values is 73.25886939873344in column VPACT
Percentage of NA values is 38.38664920045577in column LPACT
Percentage of NA values is 0.06588585469456093in column BORNINUS
Percentage of NA values is 90.61049057832278in column YRTOUS
Percentage of NA values is 39.36486036074443in column YEARMILE
Percentage of NA values is 0.0in column PROXY
Percentage of NA values is 0.0in column WHOPROXY
Percentage of NA values is 0.0in column USEPUBTR
Percentage of NA values is 83.01152614893303in column SAMEPLC
Percentage of NA values is 96.19567323716582in column W_NONE
Percentage of NA values is 96.24101820774973in column W_CANE
Percentage of NA values is 97.46765779662199in column W_WLKR
Percentage of NA values is 99.91124787808792in column W_WHCANE
Percentage of NA values is 99.93760221996574in column W_DOG
Percentage of NA values is 99.77637565788964in column W_CRUTCH
Percentage of NA values is 99.49151622729845in column W_SCOOTR
Percentage of NA values is 98.92954864313896in column W_CHAIR
Percentage of NA values is 99.66591996031346in column W_MTRCHR
Percentage of NA values is 10.405701839378038in column WORKER
Percentage of NA values is 0.0in column DIARY
Percentage of NA values is 0.0in column OUTCNTRY
Percentage of NA values is 0.0in column FRSTHM17
Percentage of NA values is 0.0in column CNTTDTR
Percentage of NA values is 59.740642270814114in column GCDWORK
Percentage of NA values is 59.75459456945532in column WKSTFIPS
Percentage of NA values is 9.373231739929153in column DRIVER
Percentage of NA values is 0.0in column OUTOF

In [14]:
#No Need to Run this if previous block is executed

#pernacols = ['1', 'PAYPROF', 'GT1JBLWK', 'WRK_HOME', 'WKFTPT', 'WRKTRANS', 'LSTTRDAY17', 'OCCAT', 'SCHTYP', 'BIKE4EX', 'BIKESHARE', 'MCUSED', 'CARRODE', 'TIMETOWK', 'NOCONG', 'PUBTIME', 'WRKTIME', 'WKRMHM', 'FLEXTIME', 'WKFMHMXX', 'SCHTRN1', 'SCHTRN2', 'MEDCOND6', 'VPACT', 'YRTOUS', 'SAMEPLC', 'W_NONE', 'W_CANE', 'W_WLKR', 'W_WHCANE', 'W_DOG', 'W_CRUTCH', 'W_SCOOTR', 'W_CHAIR', 'W_MTRCHR', 'GCDWORK', 'WKSTFIPS', 'DISTTOWK17', 'DISTTOSC17', 'WALK_DEF', 'WALK_GKQ', 'BIKE_DFR', 'BIKE_GKP', 'CONDTRAV', 'CONDRIDE', 'CONDNIGH', 'CONDRIVE', 'CONDPUB', 'CONDSPEC', 'CONDTAX']


In [15]:
#Person DataFrame after removing Columns with invalid/NA values

perpub1 = reduce(DataFrame.drop, pernacols, perpub)

print('Before removing columns with NA values: ', len(perpub.columns))
print('After removing columns with NA values: ', len(perpub1.columns))

Before removing columns with NA values: 121
After removing columns with NA values: 72

In [16]:
tripnacols = rmv_nacols(trippub)
#print(type(tripnacols))
print(len(tripnacols))
print(tripnacols)

Percentage of NA values is 0.0in column TDTRPNUM
Percentage of NA values is 0.0in column STRTTIME
Percentage of NA values is 0.0in column ENDTIME
Percentage of NA values is 0.13989238706073015in column TRVLCMIN
Percentage of NA values is 0.07138724030674343in column TRPMILES
Percentage of NA values is 0.0016627462804365708in column TRPTRANS
Percentage of NA values is 0.00033254925608731413in column TRPACCMP
Percentage of NA values is 0.0004433990081164188in column TRPHHACC
Percentage of NA values is 98.62391117831069in column TRWAITTM
Percentage of NA values is 98.61681679418083in column NUMTRANS
Percentage of NA values is 98.63621550078592in column TRACCTM
Percentage of NA values is 99.87606997723147in column DROP_PRK
Percentage of NA values is 98.63976269285085in column TREGRTM
Percentage of NA values is 13.147778238420079in column WHODROVE
Percentage of NA values is 0.02516289371060677in column WHYFROM
Percentage of NA values is 0.0in column LOOP_TRIP
Percentage of NA values is 0.0in column TRPHHVEH
Percentage of NA values is 13.147445689163995in column HHMEMDRV
Percentage of NA values is 0.0004433990081164188in column HH_ONTD
Percentage of NA values is 0.0007759482642037329in column NONHHCNT
Percentage of NA values is 0.00033254925608731413in column NUMONTRP
Percentage of NA values is 13.147556538916023in column PSGR_FLG
Percentage of NA values is 0.0in column PUBTRANS
Percentage of NA values is 0.0034363423129022456in column TRIPPURP
Percentage of NA values is 23.701118030598966in column DWELTIME
Percentage of NA values is 0.0in column TDWKND
Percentage of NA values is 33.574394594079294in column VMT_MILE
Percentage of NA values is 13.147556538916023in column DRVR_FLG
Percentage of NA values is 0.0in column WHYTRP1S
Percentage of NA values is 0.0in column ONTD_P1
Percentage of NA values is 17.421036179142067in column ONTD_P2
Percentage of NA values is 60.08799253316071in column ONTD_P3
Percentage of NA values is 75.48191929694653in column ONTD_P4
Percentage of NA values is 90.47456995838701in column ONTD_P5
Percentage of NA values is 96.739797943072in column ONTD_P6
Percentage of NA values is 98.77577533859056in column ONTD_P7
Percentage of NA values is 99.51979887420993in column ONTD_P8
Percentage of NA values is 99.80168979361993in column ONTD_P9
Percentage of NA values is 99.90865980432801in column ONTD_P10
Percentage of NA values is 99.9896909730613in column ONTD_P11
Percentage of NA values is 99.99445751239855in column ONTD_P12
Percentage of NA values is 99.99866980297566in column ONTD_P13
Percentage of NA values is 0.0in column TDCASEID
Percentage of NA values is 98.84184179079992in column TRACC_WLK
Percentage of NA values is 98.84184179079992in column TRACC_POV
Percentage of NA values is 98.84184179079992in column TRACC_BUS
Percentage of NA values is 98.84184179079992in column TRACC_CRL
Percentage of NA values is 98.84184179079992in column TRACC_SUB
Percentage of NA values is 98.84184179079992in column TRACC_OTH
Percentage of NA values is 98.84184179079992in column TREGR_WLK
Percentage of NA values is 98.84184179079992in column TREGR_POV
Percentage of NA values is 98.84184179079992in column TREGR_BUS
Percentage of NA values is 98.84184179079992in column TREGR_CRL
Percentage of NA values is 98.84184179079992in column TREGR_SUB
Percentage of NA values is 98.84184179079992in column TREGR_OTH
Percentage of NA values is 0.02449779519843214in column WHYTO
Percentage of NA values is 0.0in column TRAVDAY
Percentage of NA values is 0.017957659828714966in column HOMEOWN
Percentage of NA values is 0.0in column HHSIZE
Percentage of NA values is 0.0in column HHVEHCNT
Percentage of NA values is 2.65086097002401in column HHFAMINC
Percentage of NA values is 0.0in column DRVRCNT
Percentage of NA values is 0.0in column HHSTATE
Percentage of NA values is 0.0in column HHSTFIPS
Percentage of NA values is 0.0in column NUMADLT
Percentage of NA values is 0.0in column WRKCOUNT
Percentage of NA values is 0.0in column TDAYDATE
Percentage of 

In [17]:
#Results of previous block are hardcoded here to save time everytime we execute this project

tripnacols = ['TRWAITTM', 'NUMTRANS', 'TRACCTM', 'DROP_PRK', 'TREGRTM', 'ONTD_P3', 'ONTD_P4', 'ONTD_P5', 'ONTD_P6', 'ONTD_P7', 'ONTD_P8', 'ONTD_P9', 'ONTD_P10',
              'ONTD_P11','ONTD_P12', 'ONTD_P13', 'TRACC_WLK', 'TRACC_POV', 'TRACC_BUS', 'TRACC_CRL', 'TRACC_SUB', 'TRACC_OTH', 'TREGR_WLK','TREGR_POV','TREGR_BUS',
              'TREGR_CRL','TREGR_SUB', 'TREGR_OTH']
print(len(tripnacols))

28

In [18]:
#Trip DataFrame after removing Columns with invalid/NA values

trippub1 = reduce(DataFrame.drop, tripnacols, trippub)
print('Before removing columns with NA values: ', len(trippub.columns))
print('After removing columns with NA values: ', len(trippub1.columns))

Before removing columns with NA values: 105
After removing columns with NA values: 77

In [19]:
# Function to Drop Common Columns in DF2

def drop_comcols(df1,df2):
    a = df1.columns
    b = df2.columns
    res = df2
    #temp = ['1']
    for j in b:
        if (j == 'HOUSEID') or (j == 'PERSONID') or (j == 'VEHID'):
            #print('pass: '+ j)
            continue
        else:
            #print('ref: '+ j)
            for i in a:
                if (i == j):
                    if (i != 'HOUSEID') or (i != 'PERSONID') or (i != 'VEHID'):
                        #print(j)
                        res = res.drop(j)
                        #temp.append(j)
    #print(len(temp))
    #return temp
    #print(len(res.columns))
    #print(len(df2.columns))
    return res

In [20]:
#Drop common Columns in Person DF in Ref: Household

#Deleting the common columns (except IDs) between household dataset and person dataset
perpub_df1 = drop_comcols(hhpub1, perpub1)

#print(len(perpub_df1.columns))
#print(perpub_df1.columns)

print("Person DF before dropping common columns: ",len(perpub1.columns))
print("Person DF after dropping common columns: ",len(perpub_df1.columns))

#testing the function drop_comcols
#temp = drop_comcols(hhpub, perpub)
#print(temp)
#print(len(temp))

Person DF before dropping common columns: 72
Person DF after dropping common columns: 38

In [21]:
# Merging household and Person Dataset 
hhper_df = perpub_df1.join(hhpub1, on = ['HOUSEID'])

In [22]:
print(hhpub1.count(),len(hhpub1.columns))
print(perpub_df1.count(),len(perpub_df1.columns))
print(hhper_df.count(),len(hhper_df.columns))

126638 58
258022 38
256184 95

In [23]:
## Verification of Joins ##


# logic required to check why the resultant dataframe hhper_df has 1036 rows less than person dataframe
#Find HouseIDs that are not in merged dataset: hhper_df
print(hhpub1.select(fn.countDistinct('HOUSEID')).collect())
print(perpub_df1.select(fn.countDistinct('HOUSEID')).collect())
print(hhper_df.select(fn.countDistinct('HOUSEID')).collect())

#Difference between 2 lists
def Diff(li1, li2): 
    res = list(set(li1) - set(li2))
    return res

a = hhpub1.select("HOUSEID").collect()
b = perpub_df1.select("HOUSEID").collect()
c = hhper_df.select("HOUSEID").collect()

res = Diff(a, b)
print('Number of HouseIDs NOT common between household ds and person ds is: ' + str(len(res)))

res1 = Diff(a,c)
print('Number of HouseIDs NOT common between household ds and mergerd ds is: ' + str(len(res1)))

res2 = Diff(b,c)
print('Number of HouseIDs NOT common between person ds and merged ds is: ' + str(len(res2)))

[Row(count(DISTINCT HOUSEID)=126638)]
[Row(count(DISTINCT HOUSEID)=127569)]
[Row(count(DISTINCT HOUSEID)=126559)]
Number of HouseIDs NOT common between household ds and person ds is: 79
Number of HouseIDs NOT common between household ds and mergerd ds is: 79
Number of HouseIDs NOT common between person ds and merged ds is: 1010

In [24]:
#Deleting the common columns (except IDs) between vehicle dataset and merged dataset
vehpub_df1 = drop_comcols(hhper_df, vehpub1)

print("Vehicle DF before dropping columns: ",len(vehpub1.columns))
print("Vehicle DF after dropping columns: ",len(vehpub_df1.columns))

Vehicle DF before dropping columns: 50
Vehicle DF after dropping columns: 16

In [25]:
#Merging household and Vehicle Dataset
hhperveh_df = hhper_df.join(vehpub_df1, on = ['HOUSEID','PERSONID'])

In [26]:
print(vehpub_df1.count(),len(vehpub_df1.columns))
print(hhper_df.count(),len(hhper_df.columns))
print(hhperveh_df.count(),len(hhperveh_df.columns))

250091 16
256184 95
243980 109

In [27]:
trippub_df1 = drop_comcols(hhperveh_df, trippub1)

print("Trip DF before dropping columns: ",len(trippub1.columns))
print("Trip DF after dropping columns: ",len(trippub_df1.columns))

Trip DF before dropping columns: 77
Trip DF after dropping columns: 34

In [28]:
#Merging household and Vehicle Dataset
hhpervehtrip_df = hhperveh_df.join(trippub_df1, on = ['HOUSEID','PERSONID', 'VEHID'])

In [29]:
print(trippub_df1.count(),len(trippub_df1.columns))
print(hhperveh_df.count(),len(hhperveh_df.columns))
print(hhpervehtrip_df.count(),len(hhpervehtrip_df.columns))

902122 34
243980 109
558430 140

In [32]:
drop_columns = ['HOUSEID', 'PERSONID', 'VEHID', 'TDCASEID', 'WTHHFIN', 'WTTRDFIN', 'WTPERFIN']
#hhpervehtrip_df.drop(i for i in c)

#from functools import reduce
#from pyspark.sql import DataFrame
redmergeddf = reduce(DataFrame.drop, drop_columns, hhpervehtrip_df)

In [33]:
print(len(redmergeddf.columns))
print(redmergeddf.columns)


133
['R_AGE', 'EDUC', 'R_HISP', 'R_RELAT', 'R_SEX', 'R_RACE', 'PRMACT', 'NWALKTRP', 'WALK4EX', 'NBIKETRP', 'PTUSED', 'CARSHARE', 'RIDESHARE', 'DELIVER', 'MEDCOND', 'HEALTH', 'PHYACT', 'LPACT', 'BORNINUS', 'YEARMILE', 'PROXY', 'WHOPROXY', 'USEPUBTR', 'WORKER', 'DIARY', 'OUTCNTRY', 'FRSTHM17', 'CNTTDTR', 'DRIVER', 'OUTOFTWN', 'R_AGE_IMP', 'R_SEX_IMP', 'ALT_16', 'ALT_23', 'ALT_45', 'TRAVDAY', 'SAMPSTRAT', 'HOMEOWN', 'HHSIZE', 'HHVEHCNT', 'HHFAMINC', 'PC', 'SPHONE', 'TAB', 'WALK', 'BIKE', 'CAR', 'TAXI', 'BUS', 'TRAIN', 'PARA', 'PRICE', 'PLACE', 'WALK2SAVE', 'BIKE2SAVE', 'PTRANS', 'HHRELATD', 'DRVRCNT', 'CNTTDHH', 'HHSTATE', 'HHSTFIPS', 'NUMADLT', 'YOUNGCHILD', 'WRKCOUNT', 'TDAYDATE', 'HHRESP', 'LIF_CYC', 'MSACAT', 'MSASIZE', 'RAIL', 'URBAN', 'URBANSIZE', 'URBRUR', 'SCRESP', 'CENSUS_D', 'CENSUS_R', 'CDIVMSAR', 'HH_RACE', 'HH_HISP', 'HH_CBSA', 'RESP_CNT', 'WEBUSE17', 'SMPLSRCE', 'HBHUR', 'HTHTNRNT', 'HTPPOPDN', 'HTRESDN', 'HTEEMPDN', 'HBHTNRNT', 'HBPPOPDN', 'HBRESDN', 'VEHYEAR', 'VEHAGE', 'MAKE', 'MODEL', 'FUELTYPE', 'VEHTYPE', 'WHOMAIN', 'OD_READ', 'VEHOWNED', 'ANNMILES', 'HYBRID', 'BESTMILE', 'BEST_FLG', 'TDTRPNUM', 'STRTTIME', 'ENDTIME', 'TRVLCMIN', 'TRPMILES', 'TRPTRANS', 'TRPACCMP', 'TRPHHACC', 'WHODROVE', 'WHYFROM', 'LOOP_TRIP', 'TRPHHVEH', 'HHMEMDRV', 'HH_ONTD', 'NONHHCNT', 'NUMONTRP', 'PSGR_FLG', 'PUBTRANS', 'TRIPPURP', 'DWELTIME', 'TDWKND', 'VMT_MILE', 'DRVR_FLG', 'WHYTRP1S', 'ONTD_P1', 'ONTD_P2', 'WHYTO', 'GASPRICE', 'WHYTRP90']

In [34]:
intuitrmved = ["EDUC"
,"R_RELAT"
,"PAYPROF"
,"GT1JBLWK"
,"WRK_HOME"
,"WKFTPT"
,"WALK4EX"
,"CARRODE"
,"WRKTIME"
,"WKRMHM"
,"FLEXTIME"
,"DELIVER"
,"HEALTH"
,"VPACT"
,"LPACT"
,"BORNINUS"
,"YEARMILE"
,"PROXY"
,"WHOPROXY"
,"USEPUBTR"
,"WORKER"
,"DIARY"
,"OUTCNTRY"
,"FRSTHM17"
,"CNTTDTR"
,"GCDWORK"
,"WKSTFIPS"
,"DRIVER"
,"OUTOFTWN"
,"R_AGE_IMP"
,"R_SEX_IMP"
,"SAMPSTRAT"
,"HOMEOWN"
,"HHSIZE"
,"HHVEHCNT"
,"HHFAMINC"
,"PC"
,"SPHONE"
,"TAB"
,"PARA"
,"PRICE"
,"HHRELATD"
,"DRVRCNT"
,"CNTTDHH"
,"HHSTATE"
,"HHSTFIPS"
,"NUMADLT"
,"YOUNGCHILD"
,"WRKCOUNT"
,"HHRESP"
,"LIF_CYC"
,"MSACAT"
,"MSASIZE"
,"RAIL"
,"URBAN"
,"URBANSIZE"
,"SCRESP"
,"CENSUS_D"
,"CENSUS_R"
,"HH_RACE"
,"HH_HISP"
,"RESP_CNT"
,"WEBUSE17"
,"SMPLSRCE"
,"HBHUR"
,"HTHTNRNT"
,"HTPPOPDN"
,"HTRESDN"
,"HTEEMPDN"
,"HBHTNRNT"
,"HBPPOPDN"
,"HBRESDN"
,"VEHYEAR"
,"MAKE"
,"MODEL"
,"FUELTYPE"
,"VEHTYPE"
,"WHOMAIN"
,"OD_READ"
,"VEHOWNED"
,"ANNMILES"
,"HYBRID"
,"BEST_FLG"
,"TDTRPNUM"
,"TRPACCMP"
,"TRPHHACC"
,"WHODROVE"
,"WHYFROM"
,"LOOP_TRIP"
,"TRPHHVEH"
,"HHMEMDRV"
,"HH_ONTD"
,"NONHHCNT"
,"PSGR_FLG"
,"PUBTRANS"
,"TRIPPURP"
,"DWELTIME"
,"TDWKND"
,"VMT_MILE"
,"ONTD_P1"
,"ONTD_P2"
,"ONTD_P3"
,"WHYTO"
,"WHYTRP90"
,"HH_CBSA"
,"ALT_16"
,"ALT_23"
,"ALT_45"
,"NWALKTRP"
,"NBIKETRP"
,"WALK2SAVE"
,"BIKE2SAVE"
,"PRMACT"
,"NUMONTRP"
,"ENDTIME"]


In [35]:
print(len(intuitrmved))
print(len(redmergeddf.columns ))


115
133

In [36]:
final_df = reduce(DataFrame.drop, intuitrmved, redmergeddf)

In [37]:
print(len(final_df.columns))
final_df.printSchema()

30
root
-- R_AGE: long (nullable = true)
-- R_HISP: long (nullable = true)
-- R_SEX: long (nullable = true)
-- R_RACE: long (nullable = true)
-- PTUSED: long (nullable = true)
-- CARSHARE: long (nullable = true)
-- RIDESHARE: long (nullable = true)
-- MEDCOND: long (nullable = true)
-- PHYACT: long (nullable = true)
-- TRAVDAY: long (nullable = true)
-- WALK: long (nullable = true)
-- BIKE: long (nullable = true)
-- CAR: long (nullable = true)
-- TAXI: long (nullable = true)
-- BUS: long (nullable = true)
-- TRAIN: long (nullable = true)
-- PLACE: long (nullable = true)
-- PTRANS: long (nullable = true)
-- TDAYDATE: long (nullable = true)
-- URBRUR: long (nullable = true)
-- CDIVMSAR: long (nullable = true)
-- VEHAGE: long (nullable = true)
-- BESTMILE: double (nullable = true)
-- STRTTIME: long (nullable = true)
-- TRVLCMIN: long (nullable = true)
-- TRPMILES: double (nullable = true)
-- TRPTRANS: long (nullable = true)
-- DRVR_FLG: long (nullable = true)
-- WHYTRP1S: long (nullable = true)
-- GASPRICE: double (nullable = true)

In [38]:
#Select only rows with trips of travel where CAR, SUV, Van, Pickup Truck
#Select only rows where travel mins is known
#Consider rows where respondent is the one who was on trip
cardf = final_df.where((fn.col('TRPTRANS').isin({3,4,5,6})) & (fn.col('TRVLCMIN') > 0) & (fn.col('DRVR_FLG') == 1) & (fn.col('TRPMILES') < 30) & (fn.col('TRPMILES') >= 0) & (fn.col('TRVLCMIN') < 50))
print(cardf.count(), len(cardf.columns))


491007 30

In [39]:
b = ['TRPTRANS','DRVR_FLG']
cardf = reduce(DataFrame.drop, b, cardf)
print(cardf.count(), len(cardf.columns))
cardf.printSchema()

491007 28
root
-- R_AGE: long (nullable = true)
-- R_HISP: long (nullable = true)
-- R_SEX: long (nullable = true)
-- R_RACE: long (nullable = true)
-- PTUSED: long (nullable = true)
-- CARSHARE: long (nullable = true)
-- RIDESHARE: long (nullable = true)
-- MEDCOND: long (nullable = true)
-- PHYACT: long (nullable = true)
-- TRAVDAY: long (nullable = true)
-- WALK: long (nullable = true)
-- BIKE: long (nullable = true)
-- CAR: long (nullable = true)
-- TAXI: long (nullable = true)
-- BUS: long (nullable = true)
-- TRAIN: long (nullable = true)
-- PLACE: long (nullable = true)
-- PTRANS: long (nullable = true)
-- TDAYDATE: long (nullable = true)
-- URBRUR: long (nullable = true)
-- CDIVMSAR: long (nullable = true)
-- VEHAGE: long (nullable = true)
-- BESTMILE: double (nullable = true)
-- STRTTIME: long (nullable = true)
-- TRVLCMIN: long (nullable = true)
-- TRPMILES: double (nullable = true)
-- WHYTRP1S: long (nullable = true)
-- GASPRICE: double (nullable = true)

In [40]:
#Code to replace negative values in categorical variables

a = cardf.columns
b = ['TDAYDATE', 'URBRUR', 'CDIVMSAR', 'STRTTIME', 'TRPTRANS', 'WHYTRP1S', 'GASPRICE']
#print(len(a))
c = list(set(a) - set(b))
#print(len(c))

for i in c:
  cardf = cardf.withColumn(i, when(fn.col(i) < 0, lit(-1)).otherwise(fn.col(i)))

In [41]:
#display(cardf)
cardf.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").mode('overwrite').save("dbfs:/FileStore/tables/finalproject.csv")

# Feature Selection

In [43]:
# File location and type
file_location = "/FileStore/tables/finalproject.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
cardf = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

In [44]:
training_df, validation_df, testing_df = cardf.randomSplit([0.6, 0.3, 0.1], )

[training_df.count(), validation_df.count(), testing_df.count()]

Out[3]: [294969, 147125, 48913]

In [45]:
pipe_lr = Pipeline(stages=[StringIndexer(inputCol='R_HISP',outputCol='R_HISP_str',stringOrderType='alphabetAsc', handleInvalid="keep"),
                           StringIndexer(inputCol='R_SEX',outputCol='R_SEX_str',stringOrderType='alphabetAsc', handleInvalid="keep"),
                           StringIndexer(inputCol='R_RACE',outputCol='R_RACE_str',stringOrderType='alphabetAsc', handleInvalid="keep"),
                           StringIndexer(inputCol='MEDCOND',outputCol='MEDCOND_str',stringOrderType='alphabetAsc', handleInvalid="keep"),
                           StringIndexer(inputCol='PHYACT',outputCol='PHYACT_str',stringOrderType='alphabetAsc', handleInvalid="keep"),
                           StringIndexer(inputCol='TRAVDAY',outputCol='TRAVDAY_str',stringOrderType='alphabetAsc', handleInvalid="keep"),
                           StringIndexer(inputCol='WALK',outputCol='WALK_str',stringOrderType='alphabetAsc', handleInvalid="keep"),
                           StringIndexer(inputCol='BIKE',outputCol='BIKE_str',stringOrderType='alphabetAsc', handleInvalid="keep"),
                           StringIndexer(inputCol='CAR',outputCol='CAR_str',stringOrderType='alphabetAsc', handleInvalid="keep"),
                           StringIndexer(inputCol='TAXI',outputCol='TAXI_str',stringOrderType='alphabetAsc', handleInvalid="keep"),
                           StringIndexer(inputCol='BUS',outputCol='BUS_str',stringOrderType='alphabetAsc', handleInvalid="keep"),
                           StringIndexer(inputCol='TRAIN',outputCol='TRAIN_str',stringOrderType='alphabetAsc', handleInvalid="keep"),
                           StringIndexer(inputCol='PLACE',outputCol='PLACE_str',stringOrderType='alphabetAsc', handleInvalid="keep"),
                           StringIndexer(inputCol='PTRANS',outputCol='PTRANS_str',stringOrderType='alphabetAsc', handleInvalid="keep"),
                           StringIndexer(inputCol='URBRUR',outputCol='URBRUR_str',stringOrderType='alphabetAsc', handleInvalid="keep"),
                           StringIndexer(inputCol='CDIVMSAR',outputCol='CDIVMSAR_str',stringOrderType='alphabetAsc', handleInvalid="keep"),
                           StringIndexer(inputCol='WHYTRP1S',outputCol='WHYTRP1S_str',stringOrderType='alphabetAsc', handleInvalid="keep"),
                           OneHotEncoder(inputCol='R_HISP_str',outputCol='R_HISP_ohe',dropLast=False),
                           OneHotEncoder(inputCol='R_SEX_str',outputCol='R_SEX_ohe',dropLast=False),
                           OneHotEncoder(inputCol='R_RACE_str',outputCol='R_RACE_ohe',dropLast=False),
                           OneHotEncoder(inputCol='MEDCOND_str',outputCol='MEDCOND_ohe',dropLast=False),
                           OneHotEncoder(inputCol='PHYACT_str',outputCol='PHYACT_ohe',dropLast=False),
                           OneHotEncoder(inputCol='TRAVDAY_str',outputCol='TRAVDAY_ohe',dropLast=False),
                           OneHotEncoder(inputCol='WALK_str',outputCol='WALK_ohe',dropLast=False),
                           OneHotEncoder(inputCol='BIKE_str',outputCol='BIKE_ohe',dropLast=False),
                           OneHotEncoder(inputCol='CAR_str',outputCol='CAR_ohe',dropLast=False),
                           OneHotEncoder(inputCol='TAXI_str',outputCol='TAXI_ohe',dropLast=False),
                           OneHotEncoder(inputCol='BUS_str',outputCol='BUS_ohe',dropLast=False),
                           OneHotEncoder(inputCol='TRAIN_str',outputCol='TRAIN_ohe',dropLast=False),
                           OneHotEncoder(inputCol='PLACE_str',outputCol='PLACE_ohe',dropLast=False),
                           OneHotEncoder(inputCol='PTRANS_str',outputCol='PTRANS_ohe',dropLast=False),
                           OneHotEncoder(inputCol='URBRUR_str',outputCol='URBRUR_ohe',dropLast=False),
                           OneHotEncoder(inputCol='CDIVMSAR_str',outputCol='CDIVMSAR_ohe',dropLast=False),
                           OneHotEncoder(inputCol='WHYTRP1S_str',outputCol='WHYTRP1S_ohe',dropLast=False),
                           feature.VectorAssembler(inputCols=['R_AGE', 'PTUSED', 'CARSHARE','RIDESHARE', 'TDAYDATE','VEHAGE','BESTMILE','STRTTIME',
                                                              'TRPMILES', 'GASPRICE']
                                                   ,outputCol='num_features'),
                           feature.StandardScaler(withMean=True,inputCol='num_features', outputCol='zfeatures'),
                           feature.VectorAssembler(inputCols=['R_HISP_ohe','R_SEX_ohe', 'R_RACE_ohe', 'MEDCOND_ohe','PHYACT_ohe', 'TRAVDAY_ohe', 'WALK_ohe', 
                                                              'BIKE_ohe',  'CAR_ohe', 'TAXI_ohe', 'BUS_ohe', 'TRAIN_ohe', 'PLACE_ohe','PTRANS_ohe', 'URBRUR_ohe',
                                                              'CDIVMSAR_ohe', 'WHYTRP1S_ohe', 'zfeatures'
                                                             ],
                                                   outputCol='features')
                           ])

In [46]:
lr = regression.LinearRegression(featuresCol='features', labelCol='TRVLCMIN')
pipe1_model = Pipeline(stages=[pipe_lr, lr]).fit(training_df)

In [47]:
rmse = fn.sqrt(fn.mean((fn.col('TRVLCMIN') - fn.col('prediction'))**2)).alias('rmse')

In [48]:
pipe1_model.transform(training_df).select(rmse).show()
pipe1_model.transform(validation_df).select(rmse).show()
pipe1_model.transform(testing_df).select(rmse).show()

+------------------+
 rmse|
+------------------+
6.0481989927212805|
+------------------+

+-----------------+
 rmse|
+-----------------+
6.091620576531878|
+-----------------+

+-----------------+
 rmse|
+-----------------+
6.027741962397137|
+-----------------+

In [49]:
pipe1_model.stages[-1].coefficients

Out[9]: DenseVector([-0.1429, 0.3285, -0.3211, 0.0, 0.8291, -0.0176, 0.0162, 0.0, -0.2558, -0.4001, 0.6239, 0.5955, -0.0227, 0.5695, -0.2158, 0.008, 0.0, -0.5221, 0.319, -0.316, 0.0, 0.3577, 0.0586, 0.0266, -0.0538, 0.0, -0.4082, 0.0161, 0.0487, 0.0535, 0.0458, 0.1404, -0.1765, 0.0, 0.1824, 0.2737, 0.1563, -0.0334, -0.2377, -0.1226, 0.0, 0.0766, -0.2432, 0.0055, -0.0219, -0.0714, 0.0303, 0.0, 0.1352, -0.1512, 0.0763, 0.594, 0.8229, 1.3772, 0.0, -0.0828, -0.1549, 0.6598, 0.5341, 0.0769, -0.1344, 0.0, -0.3368, 0.1929, 0.4618, 0.7, 0.2336, -0.074, 0.0, 0.1764, 0.155, 0.2689, 0.2175, -0.152, 0.0128, 0.0, 0.1312, 0.4618, 0.1644, -0.0822, -0.2185, -0.1843, 0.0, 0.2042, 0.061, 0.1032, -0.011, 0.0585, -0.0959, 0.0, 0.3997, -0.3997, 0.0, 0.3083, 0.6615, 0.4696, -0.2354, 0.71, 0.0422, -0.1884, -1.1853, 0.5558, -0.1099, -0.8408, -1.5752, -4.8248, -0.356, -0.5771, -1.803, 1.2659, 0.2752, 0.2323, -0.845, 0.4135, -0.1233, -1.3993, -4.313, 0.4835, -0.4195, -1.8375, -3.2681, 0.2847, 0.0943, -1.0446, 1.445, 0.5462, -0.2372, -0.7555, 0.0, 0.2266, 0.2682, 0.5509, 1.6464, -0.5157, 0.2589, -0.4486, -0.3891, -0.0664, 0.0, 0.3729, 0.0071, 0.0186, 0.0658, 0.0354, 0.0784, -0.0553, 0.2665, 7.7655, -0.07])

In [50]:
#Elastic Net Regression

lambda_par = 0.02
alpha_par = 0.3
en_lr = LinearRegression().\
        setLabelCol('TRVLCMIN').\
        setFeaturesCol('features').\
        setRegParam(lambda_par).\
        setMaxIter(100).\
        setElasticNetParam(alpha_par)

In [51]:
en_lr_estimator = Pipeline(stages=[pipe_lr, en_lr])

In [52]:
grid = ParamGridBuilder().\
    addGrid(en_lr.regParam, [0., 0.01, 0.02]).\
    addGrid(en_lr.elasticNetParam, [0., 0.2, 0.4]).\
    build()

In [53]:
grid

Out[13]: [{Param(parent='LinearRegression_e2968d429d40', name='regParam', doc='regularization parameter (>= 0).'): 0.0,
 Param(parent='LinearRegression_e2968d429d40', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0},
 {Param(parent='LinearRegression_e2968d429d40', name='regParam', doc='regularization parameter (>= 0).'): 0.0,
 Param(parent='LinearRegression_e2968d429d40', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.2},
 {Param(parent='LinearRegression_e2968d429d40', name='regParam', doc='regularization parameter (>= 0).'): 0.0,
 Param(parent='LinearRegression_e2968d429d40', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.4},
 {Param(parent='LinearRegression_e2968d429d40', name='regParam', doc='regularization parameter (>= 0).'): 0.01,
 Param(parent='LinearRegression_e2968d429d40', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0},
 {Param(parent='LinearRegression_e2968d429d40', name='regParam', doc='regularization parameter (>= 0).'): 0.01,
 Param(parent='LinearRegression_e2968d429d40', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.2},
 {Param(parent='LinearRegression_e2968d429d40', name='regParam', doc='regularization parameter (>= 0).'): 0.01,
 Param(parent='LinearRegression_e2968d429d40', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.4},
 {Param(parent='LinearRegression_e2968d429d40', name='regParam', doc='regularization parameter (>= 0).'): 0.02,
 Param(parent='LinearRegression_e2968d429d40', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0},
 {Param(parent='LinearRegression_e2968d429d40', name='regParam', doc='regularization parameter (>= 0).'): 0.02,
 Param(parent='LinearRegression_e2968d429d40', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.2},
 {Param(parent='LinearRegression_e2968d429d40', name='regParam', doc='regularization parameter (>= 0).'): 0.02,
 Param(parent='LinearRegression_e2968d429d40', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.4}]

In [54]:
all_models = []
for j in range(len(grid)):
    print("Fitting model {}".format(j+1))
    model = en_lr_estimator.fit(training_df, grid[j])
    all_models.append(model)

Fitting model 1
Fitting model 2
Fitting model 3
Fitting model 4
Fitting model 5
Fitting model 6
Fitting model 7
Fitting model 8
Fitting model 9

In [55]:
t1 = all_models[0].transform(validation_df).select(rmse).show()
t2 = all_models[1].transform(validation_df).select(rmse).show()
t3 = all_models[2].transform(validation_df).select(rmse).show()
t4 = all_models[3].transform(validation_df).select(rmse).show()
t5 = all_models[4].transform(validation_df).select(rmse).show()
t6= all_models[5].transform(validation_df).select(rmse).show()
t7 = all_models[6].transform(validation_df).select(rmse).show()
t8 = all_models[7].transform(validation_df).select(rmse).show()
t9 = all_models[8].transform(validation_df).select(rmse).show()

+-----------------+
 rmse|
+-----------------+
6.091620576531878|
+-----------------+

+-----------------+
 rmse|
+-----------------+
6.091620576531875|
+-----------------+

+-----------------+
 rmse|
+-----------------+
6.091620576531878|
+-----------------+

+-----------------+
 rmse|
+-----------------+
6.091633015240814|
+-----------------+

+------------------+
 rmse|
+------------------+
6.0915525463092965|
+------------------+

+-----------------+
 rmse|
+-----------------+
6.091634277792081|
+-----------------+

+----------------+
 rmse|
+----------------+
6.09165674432519|
+----------------+

+-----------------+
 rmse|
+-----------------+
6.091668824129739|
+-----------------+

+-----------------+
 rmse|
+-----------------+
6.092022580997208|
+-----------------+

In [56]:
best_model = all_models[8]

In [57]:
best_model.stages[-1].coefficients

Out[17]: DenseVector([0.0, 0.529, -0.0919, 0.0, 0.4435, -0.0143, 0.0, 0.0, -0.0872, -0.3836, 0.6313, 0.5873, 0.0, 0.4084, -0.148, 0.0, 0.0, 0.0, 0.5072, -0.1097, 0.0, 0.0951, 0.0082, 0.0, -0.0707, 0.0, -0.418, -0.0073, 0.0, 0.0, 0.0, 0.0804, -0.193, 0.0, 0.1133, 0.2776, 0.1655, 0.0, -0.191, -0.0645, 0.0, 0.0, -0.1631, 0.0, 0.0, -0.0662, 0.012, 0.0, 0.0, -0.2284, 0.0, 0.4349, 0.4911, 1.146, 0.0, 0.0, 0.0, 0.565, 0.4682, 0.0207, -0.1661, 0.0, -0.028, 0.2184, 0.4972, 0.7503, 0.2833, 0.0, 0.0, 0.0, 0.0978, 0.2267, 0.2064, -0.1118, 0.0, 0.0, 0.1784, 0.5152, 0.2305, 0.0, -0.1231, -0.0768, 0.0, 0.1206, 0.0133, 0.0535, -0.0384, 0.0, -0.1369, 0.0, 0.4023, -0.4023, 0.0, 0.2387, 0.3883, 0.4143, -0.0395, 0.7017, 0.0081, -0.1249, -1.0952, 0.5438, 0.0, -0.7191, -1.4355, -2.2412, -0.1535, -0.4474, -1.5918, 1.2876, 0.2937, 0.2948, -0.7166, 0.3471, 0.0, -1.0901, -2.6421, 0.5995, -0.2514, -1.6322, -1.009, 0.18, 0.0406, -0.9195, 1.3685, 0.4653, -0.2543, -0.7477, 0.0, 0.0, 0.0099, 0.2631, 1.3698, -0.7305, 0.0, -0.6481, -0.5936, -0.2332, 0.0, 0.3579, 0.006, 0.012, 0.062, 0.0114, 0.0698, -0.0499, 0.2531, 7.7488, 0.0])

In [58]:
features = best_model.stages[0].stages[-3].getInputCols()

for i in range(17):
  for j in range(len(best_model.stages[0].stages[i].labels)):
    a = best_model.stages[0].stages[i].getInputCol()+best_model.stages[0].stages[i].labels[j]
    features.append(a)

In [59]:
featureimp = pd.DataFrame(list(zip(features,best_model.stages[-1].coefficients.toArray())),columns = ['features', 'weights']).sort_values('weights', ascending = False)

In [60]:
featureimp

,features,weights
124,WHYTRP1S10,1.368506
109,CDIVMSAR63,1.287555
53,CAR3,1.146040
65,BUS3,0.750252
97,CDIVMSAR32,0.701699
10,R_HISP-1,0.631274
117,CDIVMSAR83,0.599482
11,R_HISP1,0.587347
57,TAXI1,0.565048
101,CDIVMSAR42,0.543750


In [61]:
#Visualizations
plt.style.use('ggplot')
rc('figure', figsize=(20,10))
fig, ax = plt.subplots()

featureimp.plot.bar(x = 'features', y = 'weights', ax = ax)
ax.grid(False)
ax.set_facecolor((1, 1, 1))
ax.spines['bottom'].set_color('black')
ax.spines['left'].set_color('black')
fig.suptitle('Elastic Net Regularization', fontsize=15)
plt.ylabel("Feature importance")
ax.legend_.remove()
display(plt.show())

In [62]:
pipe_final = Pipeline(stages=[
                           StringIndexer(inputCol='TRAVDAY',outputCol='TRAVDAY_str',stringOrderType='alphabetAsc', handleInvalid="keep"), 
                           StringIndexer(inputCol='URBRUR',outputCol='URBRUR_str',stringOrderType='alphabetAsc', handleInvalid="keep"),
                           StringIndexer(inputCol='CDIVMSAR',outputCol='CDIVMSAR_str',stringOrderType='alphabetAsc', handleInvalid="keep"),
                           StringIndexer(inputCol='WHYTRP1S',outputCol='WHYTRP1S_str',stringOrderType='alphabetAsc', handleInvalid="keep"),
                           OneHotEncoder(inputCol='TRAVDAY_str',outputCol='TRAVDAY_ohe',dropLast=False),
                           OneHotEncoder(inputCol='URBRUR_str',outputCol='URBRUR_ohe',dropLast=False),
                           OneHotEncoder(inputCol='CDIVMSAR_str',outputCol='CDIVMSAR_ohe',dropLast=False),
                           OneHotEncoder(inputCol='WHYTRP1S_str',outputCol='WHYTRP1S_ohe',dropLast=False),
                           feature.VectorAssembler(inputCols=['R_AGE', 'STRTTIME', 'TRPMILES' ],
                                                   outputCol='num_features'),
                           feature.StandardScaler(withMean=True,inputCol='num_features', 
                                                  outputCol='zfeatures'),
                           feature.VectorAssembler(inputCols=['TRAVDAY_ohe', 'URBRUR_ohe','CDIVMSAR_ohe', 'WHYTRP1S_ohe', 'zfeatures'],
                                                   outputCol='features')
                           ])

In [63]:
lr = regression.LinearRegression(featuresCol='features', labelCol='TRVLCMIN')
pipelr_model = Pipeline(stages=[pipe_final, lr]).fit(training_df)

In [64]:
pipelr_model.transform(training_df).select(rmse).show()
pipelr_model.transform(validation_df).select(rmse).show()
pipelr_model.transform(testing_df).select(rmse).show()

+-----------------+
 rmse|
+-----------------+
6.080148106981362|
+-----------------+

+-----------------+
 rmse|
+-----------------+
6.122668671312896|
+-----------------+

+-----------------+
 rmse|
+-----------------+
6.063452276961657|
+-----------------+

In [65]:
pipe_all = Pipeline(stages=[StringIndexer(inputCol='R_HISP',outputCol='R_HISP_str',stringOrderType='alphabetAsc', handleInvalid="keep"),
                           StringIndexer(inputCol='R_SEX',outputCol='R_SEX_str',stringOrderType='alphabetAsc', handleInvalid="keep"),
                           StringIndexer(inputCol='R_RACE',outputCol='R_RACE_str',stringOrderType='alphabetAsc', handleInvalid="keep"),
                           StringIndexer(inputCol='MEDCOND',outputCol='MEDCOND_str',stringOrderType='alphabetAsc', handleInvalid="keep"),
                           StringIndexer(inputCol='PHYACT',outputCol='PHYACT_str',stringOrderType='alphabetAsc', handleInvalid="keep"),
                           StringIndexer(inputCol='TRAVDAY',outputCol='TRAVDAY_str',stringOrderType='alphabetAsc', handleInvalid="keep"),
                           StringIndexer(inputCol='WALK',outputCol='WALK_str',stringOrderType='alphabetAsc', handleInvalid="keep"),
                           StringIndexer(inputCol='BIKE',outputCol='BIKE_str',stringOrderType='alphabetAsc', handleInvalid="keep"),
                           StringIndexer(inputCol='CAR',outputCol='CAR_str',stringOrderType='alphabetAsc', handleInvalid="keep"),
                           StringIndexer(inputCol='TAXI',outputCol='TAXI_str',stringOrderType='alphabetAsc', handleInvalid="keep"),
                           StringIndexer(inputCol='BUS',outputCol='BUS_str',stringOrderType='alphabetAsc', handleInvalid="keep"),
                           StringIndexer(inputCol='TRAIN',outputCol='TRAIN_str',stringOrderType='alphabetAsc', handleInvalid="keep"),
                           StringIndexer(inputCol='PLACE',outputCol='PLACE_str',stringOrderType='alphabetAsc', handleInvalid="keep"),
                           StringIndexer(inputCol='PTRANS',outputCol='PTRANS_str',stringOrderType='alphabetAsc', handleInvalid="keep"),
                           StringIndexer(inputCol='URBRUR',outputCol='URBRUR_str',stringOrderType='alphabetAsc', handleInvalid="keep"),
                           StringIndexer(inputCol='CDIVMSAR',outputCol='CDIVMSAR_str',stringOrderType='alphabetAsc', handleInvalid="keep"),
                           StringIndexer(inputCol='WHYTRP1S',outputCol='WHYTRP1S_str',stringOrderType='alphabetAsc', handleInvalid="keep"),
                           feature.VectorAssembler(inputCols=['R_HISP_str','R_SEX_str', 'R_RACE_str', 'MEDCOND_str','PHYACT_str', 'TRAVDAY_str', 'WALK_str', 
                                                              'BIKE_str',  'CAR_str', 'TAXI_str', 'BUS_str', 'TRAIN_str', 'PLACE_str', 'PTRANS_str', 'URBRUR_str',
                                                              'CDIVMSAR_str', 'WHYTRP1S_str','R_AGE', 'PTUSED', 'CARSHARE','RIDESHARE',
                                                              'TDAYDATE','VEHAGE','BESTMILE','STRTTIME','TRPMILES', 'GASPRICE'], outputCol='features')])

In [66]:
rf_all = RandomForestRegressor(featuresCol="features", labelCol="TRVLCMIN",maxDepth = 7,numTrees=500,maxBins=36)
pipe_rf_all = Pipeline(stages=[pipe_all,rf_all]).fit(training_df)

In [67]:
#rmse on all features 
evaluator = evaluation.RegressionEvaluator(labelCol='TRVLCMIN', metricName='rmse')
rf_rmse_val = evaluator.evaluate(pipe_rf_all.transform(validation_df))
print(rf_rmse_val) 

6.03293413761985

In [68]:
model_rf_all = pipe_rf_all.stages[-1]
feature_importance= pd.DataFrame(list(zip(['R_HISP_str','R_SEX_str', 'R_RACE_str', 'MEDCOND_str','PHYACT_str', 'TRAVDAY_str', 'WALK_str','BIKE_str', 'CAR_str',
                                           'TAXI_str', 'BUS_str', 'TRAIN_str', 'PLACE_str', 'PTRANS_str', 'URBRUR_str','CDIVMSAR_str', 'WHYTRP1S_str','R_AGE',
                                           'PTUSED', 'CARSHARE','RIDESHARE','TDAYDATE','VEHAGE','BESTMILE','STRTTIME','TRPMILES', 'GASPRICE'],
                                          model_rf_all.featureImportances.toArray())),
                                 columns = ['feature', 'importance']).sort_values('importance', ascending = False)

In [69]:
print(feature_importance)

feature importance
25 TRPMILES 0.933078
16 WHYTRP1S_str 0.029194
15 CDIVMSAR_str 0.017567
24 STRTTIME 0.007844
14 URBRUR_str 0.003549
2 R_RACE_str 0.002316
17 R_AGE 0.001827
23 BESTMILE 0.000957
6 WALK_str 0.000431
12 PLACE_str 0.000317
5 TRAVDAY_str 0.000307
10 BUS_str 0.000291
13 PTRANS_str 0.000288
11 TRAIN_str 0.000283
26 GASPRICE 0.000260
1 R_SEX_str 0.000246
9 TAXI_str 0.000225
22 VEHAGE 0.000181
8 CAR_str 0.000153
18 PTUSED 0.000123
20 RIDESHARE 0.000123
7 BIKE_str 0.000109
3 MEDCOND_str 0.000093
0 R_HISP_str 0.000082
21 TDAYDATE 0.000073
4 PHYACT_str 0.000044
19 CARSHARE 0.000038

In [70]:
plt.style.use('ggplot')
rc('figure', figsize=(20,10))
fig, ax = plt.subplots()

pd.DataFrame(list(zip(['R_HISP_str','R_SEX_str', 'R_RACE_str', 'MEDCOND_str','PHYACT_str', 'TRAVDAY_str', 'WALK_str','BIKE_str',  'CAR_str', 'TAXI_str', 'BUS_str',
                       'TRAIN_str', 'PLACE_str', 'PTRANS_str', 'URBRUR_str','CDIVMSAR_str', 'WHYTRP1S_str','R_AGE', 'PTUSED','CARSHARE','RIDESHARE',
                       'TDAYDATE','VEHAGE','BESTMILE','STRTTIME','TRPMILES', 'GASPRICE'], 
                      model_rf_all.featureImportances.toArray())),
             columns = ['feature', 'importance']).sort_values('importance', ascending = False).set_index('feature')[0:29].plot(kind = 'bar', ax = ax)

ax.grid(False)
ax.set_facecolor((0.91,0.41,0.17))
ax.spines['bottom'].set_color('black')
ax.spines['left'].set_color('black')
fig.suptitle('Top 30 Feature Importances', fontsize=15)
plt.ylabel("Feature Importance")
ax.legend_.remove()
display(plt.show())

# Regression Models to Predict Commute Time

In [71]:
pipe = Pipeline(stages=[StringIndexer(inputCol='URBRUR',outputCol='URBRUR_str',stringOrderType='alphabetAsc', handleInvalid="keep"),
                           StringIndexer(inputCol='TRAVDAY',outputCol='TRAVDAY_str',stringOrderType='alphabetAsc', handleInvalid="keep"),
                           StringIndexer(inputCol='CDIVMSAR',outputCol='CDIVMSAR_str',stringOrderType='alphabetAsc', handleInvalid="keep"),
                           StringIndexer(inputCol='WHYTRP1S',outputCol='WHYTRP1S_str',stringOrderType='alphabetAsc', handleInvalid="keep"),
                           feature.VectorAssembler(inputCols=['URBRUR_str','R_AGE','TRAVDAY_str','CDIVMSAR_str', 'WHYTRP1S_str','STRTTIME','TRPMILES'], outputCol='features')]) 

In [72]:
rf_sel = RandomForestRegressor(featuresCol="features", labelCol="TRVLCMIN",maxDepth = 7,numTrees=500,maxBins=36)
pipe_rf = Pipeline(stages=[pipe,rf_sel]).fit(training_df)

In [73]:
evaluator = evaluation.RegressionEvaluator(labelCol='TRVLCMIN', metricName='rmse')

In [74]:
#Running the with cross validation
pipe_rf = Pipeline(stages=[pipe,rf_sel])

# Number of trees in random forest
n_estimators = [300,400,500,600]

# Maximum number of levels in tree
max_depth = [7,8,9]

tfparamgrid = {'n_estimators': n_estimators,
               'max_depth': max_depth}

rfcv = CrossValidator(estimator = pipe_rf,
                      estimatorParamMaps = rfparamGrid,
                      evaluator =evaluator,
                      numFolds = 10)
rfcvModel = rfcv.fit(training_df)
print(rfcvModel)
# Use test set here so we can measu the accuracy of our model on new data
rfpredictions = rfcvModel.transform(testing_df)
# cvModel uses the best model found from the Cross Validation
# Evaluate best model
print('RMSE:', rfevaluator.evaluate(rfpredictions))

In [75]:
#rrmse on all features 

rf_rmse = evaluator.evaluate(pipe_rf.transform(validation_df))
print(rf_rmse)

5.856938315544348

In [76]:
#Random Forest for 300 trees & Depth 7
rf_300_7 = RandomForestRegressor(featuresCol="features", labelCol="TRVLCMIN",maxDepth = 7,numTrees=300,maxBins=36)
pipe_rf_300 = Pipeline(stages=[pipe,rf_300_7]).fit(training_df)

rf_300_7_rmse = evaluator.evaluate(pipe_rf_300.transform(validation_df)) 
print(rf_300_7_rmse)

5.852045648906979

In [77]:
#Random Forest for 300 trees & Depth 8
rf_300 = RandomForestRegressor(featuresCol="features", labelCol="TRVLCMIN",maxDepth = 8,numTrees=300,maxBins=36)
pipe_rf_300 = Pipeline(stages=[pipe,rf_300]).fit(training_df)

rf_300_rmse = evaluator.evaluate(pipe_rf_300.transform(validation_df)) 
print(rf_300_rmse)

5.808890468014892

In [78]:
#Random Forest for 300 trees & Depth 9
rf_300_9 = RandomForestRegressor(featuresCol="features", labelCol="TRVLCMIN",maxDepth = 9,numTrees=300,maxBins=36)
pipe_rf_300 = Pipeline(stages=[pipe,rf_300_9]).fit(training_df)

rf_300_9_rmse = evaluator.evaluate(pipe_rf_300.transform(validation_df)) 
print(rf_300_9_rmse)

5.782740957152559

In [79]:
#Random Forest for 400 trees & Depth 7
rf_400 = RandomForestRegressor(featuresCol="features", labelCol="TRVLCMIN",maxDepth = 7,numTrees=400,maxBins=36)
pipe_rf_400 = Pipeline(stages=[pipe,rf_400]).fit(training_df)

rf_400_rmse = evaluator.evaluate(pipe_rf_400.transform(validation_df)) 
print(rf_400_rmse)

5.8589615154681525

In [80]:
#Random Forest for 400 trees & Depth 8
rf_400_8 = RandomForestRegressor(featuresCol="features", labelCol="TRVLCMIN",maxDepth = 8,numTrees=400,maxBins=36)
pipe_rf_400 = Pipeline(stages=[pipe,rf_400_8]).fit(training_df)

rf_400_8_rmse = evaluator.evaluate(pipe_rf_400.transform(validation_df)) 
print(rf_400_8_rmse)

5.8126283317261755

In [81]:
#Random Forest for 400 trees & Depth 9
rf_400_9 = RandomForestRegressor(featuresCol="features", labelCol="TRVLCMIN",maxDepth = 9,numTrees=400,maxBins=36)
pipe_rf_400 = Pipeline(stages=[pipe,rf_400_9]).fit(training_df)

rf_400_9_rmse = evaluator.evaluate(pipe_rf_400.transform(validation_df)) 
print(rf_400_9_rmse)

5.784460836072528

In [82]:
#Random Forest for 500 trees & Depth 7
rf_500_7 = RandomForestRegressor(featuresCol="features", labelCol="TRVLCMIN",maxDepth = 7,numTrees=500,maxBins=36)
pipe_rf_500 = Pipeline(stages=[pipe,rf_500_7]).fit(training_df)

rf_500_7_rmse = evaluator.evaluate(pipe_rf_500.transform(validation_df)) 
print(rf_500_7_rmse)

5.856938315544348

In [83]:
#Random Forest for 500 trees & Depth 8
rf_500_8 = RandomForestRegressor(featuresCol="features", labelCol="TRVLCMIN",maxDepth = 8,numTrees=500,maxBins=36)
pipe_rf_500 = Pipeline(stages=[pipe,rf_500_8]).fit(training_df)

rf_500_8_rmse = evaluator.evaluate(pipe_rf_500.transform(validation_df)) 
print(rf_500_8_rmse)

5.811822153063375

In [84]:
#Random Forest for 500 trees & Depth 9 
rf_500_9 = RandomForestRegressor(featuresCol="features", labelCol="TRVLCMIN",maxDepth = 9,numTrees=500,maxBins=36)
pipe_rf_500 = Pipeline(stages=[pipe,rf_500_9]).fit(training_df)

rf_500_9_rmse = evaluator.evaluate(pipe_rf_500.transform(validation_df)) 
print(rf_500_9_rmse)

In [85]:
#Random Forest for 600 trees & Depth 7 
rf_600_7 = RandomForestRegressor(featuresCol="features", labelCol="TRVLCMIN",maxDepth = 7,numTrees=600,maxBins=36)
pipe_rf_600 = Pipeline(stages=[pipe,rf_600_7]).fit(training_df)

rf_600_7_rmse = evaluator.evaluate(pipe_rf_600.transform(validation_df)) 
print(rf_600_7_rmse)

5.8895274849194905

In [86]:
#Random Forest for 600 trees & Depth 8 
rf_600_8 = RandomForestRegressor(featuresCol="features", labelCol="TRVLCMIN",maxDepth = 8,numTrees=600,maxBins=36)
pipe_rf_600 = Pipeline(stages=[pipe,rf_600_8]).fit(training_df)

rf_600_8_rmse = evaluator.evaluate(pipe_rf_600.transform(validation_df)) 
print(rf_600_8_rmse)

5.846500243061041

In [87]:
#Random Forest for 600 Trees & Depth 9 
rf_600_9 = RandomForestRegressor(featuresCol="features", labelCol="TRVLCMIN",maxDepth = 9,numTrees=600,maxBins=36)
pipe_rf_600 = Pipeline(stages=[pipe,rf_600_9]).fit(training_df)

rf_600_9_rmse = evaluator.evaluate(pipe_rf_600.transform(validation_df)) 
print(rf_600_9_rmse)

In [88]:
#GBT with 50 Trees and Depth 1
gbt_50_1 =GBTRegressor(featuresCol='features', labelCol='TRVLCMIN',maxBins=36,maxIter=50,maxDepth=1)
pipe_gbt_50_1 = Pipeline(stages=[pipe,gbt_50_1]).fit(training_df)

gbt_50_1_rmse = evaluator.evaluate(pipe_gbt_50_1.transform(validation_df)) 
print(gbt_50_1_rmse)

6.173378755465039

In [89]:
#GBT with 50 Trees and Depth 2
gbt_50_2 =GBTRegressor(featuresCol='features', labelCol='TRVLCMIN',maxBins=36,maxIter=50,maxDepth=2)
pipe_gbt_50_2 = Pipeline(stages=[pipe,gbt_50_2]).fit(training_df)

gbt_50_2_rmse = evaluator.evaluate(pipe_gbt_50_2.transform(validation_df)) 
print(gbt_50_2_rmse)

5.913703777000477

In [90]:
#GBT with 50 Trees and Depth 5
gbt_50_5 =GBTRegressor(featuresCol='features', labelCol='TRVLCMIN',maxBins=36,maxIter=50,maxDepth=5)
pipe_gbt_50_5 = Pipeline(stages=[pipe,gbt_50_5]).fit(training_df)

gbt_50_5_rmse = evaluator.evaluate(pipe_gbt_50_5.transform(validation_df)) 
print(gbt_50_5_rmse)

5.790977384035534

In [91]:
#GBT with 100 Trees and Depth 1
gbt_100_1 =GBTRegressor(featuresCol='features', labelCol='TRVLCMIN',maxBins=36,maxIter=100,maxDepth=1)
pipe_gbt_100_1 = Pipeline(stages=[pipe,gbt_100_1]).fit(training_df)

gbt_100_1_rmse = evaluator.evaluate(pipe_gbt_100_1.transform(validation_df)) 
print(gbt_100_1_rmse)

5.995093640830483

In [92]:
#GBT with 100 Trees and Depth 2
gbt_100_2 =GBTRegressor(featuresCol='features', labelCol='TRVLCMIN',maxBins=36,maxIter=100,maxDepth=2)
pipe_gbt_100_2 = Pipeline(stages=[pipe,gbt_100_2]).fit(training_df)

gbt_100_2_rmse = evaluator.evaluate(pipe_gbt_100_2.transform(validation_df)) 
print(gbt_100_2_rmse)

5.829991341422633

In [93]:
#GBT with 100 Trees and Depth 5
gbt_100_5 =GBTRegressor(featuresCol='features', labelCol='TRVLCMIN',maxBins=36,maxIter=100,maxDepth=5)
pipe_gbt_100_5 = Pipeline(stages=[pipe,gbt_100_5]).fit(training_df)

gbt_100_5_rmse = evaluator.evaluate(pipe_gbt_100_5.transform(validation_df)) 
print(gbt_100_5_rmse)

5.771770299881011

In [94]:
#GBT with 200 Trees and Depth 1
gbt_200_1 =GBTRegressor(featuresCol='features', labelCol='TRVLCMIN',maxBins=36,maxIter=200,maxDepth=1)
pipe_gbt_200_1 = Pipeline(stages=[pipe,gbt_200_1]).fit(training_df)

gbt_200_1_rmse = evaluator.evaluate(pipe_gbt_200_1.transform(validation_df)) 
print(gbt_200_1_rmse)

5.895921508421511

In [95]:
#GBT with 200 Trees and Depth 2
gbt_200_2 =GBTRegressor(featuresCol='features', labelCol='TRVLCMIN',maxBins=36,maxIter=200,maxDepth=2)
pipe_gbt_200_2 = Pipeline(stages=[pipe,gbt_200_2]).fit(training_df)

gbt_200_2_rmse = evaluator.evaluate(pipe_gbt_200_2.transform(validation_df)) 
print(gbt_200_2_rmse)

5.803427623494713

In [96]:
#GBT with 200 Trees and Depth 5
gbt_200_5 =GBTRegressor(featuresCol='features', labelCol='TRVLCMIN',maxBins=36,maxIter=200,maxDepth=5)
pipe_gbt_200_5 = Pipeline(stages=[pipe,gbt_200_5]).fit(training_df)

gbt_200_5_rmse = evaluator.evaluate(pipe_gbt_200_5.transform(validation_df)) 
print(gbt_200_5_rmse)

In [97]:
#Best Model is GBT with Number of Trees = 100 and Depth = 5

best_model = pipe_gbt_100_5
best_model_rmse = evaluator.evaluate(best_model.transform(testing_df)) 
print(best_model_rmse)

5.783260405035978